In [9]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import seaborn as sns
import matplotlib.patches as patches
import warnings
import datetime as dt
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import os
warnings.filterwarnings("ignore")


In [10]:
sd = pd.read_csv('sales_data_sample.csv', encoding = 'unicode_escape')

In [13]:
sd.head()

,ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,...,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE
0,10107,30,95.70,2,2871.00,2/24/2003 0:00,Shipped,1,2,2003,...,897 Long Airport Avenue,NaN,NYC,NY,10022,USA,NaN,Yu,Kwai,Small
1,10121,34,81.35,5,2765.90,5/7/2003 0:00,Shipped,2,5,2003,...,59 rue de l'Abbaye,NaN,Reims,NaN,51100,France,EMEA,Henriot,Paul,Small
2,10134,41,94.74,2,3884.34,7/1/2003 0:00,Shipped,3,7,2003,...,27 rue du Colonel Pierre Avia,NaN,Paris,NaN,75508,France,EMEA,Da Cunha,Daniel,Medium
3,10145,45,83.26,6,3746.70,8/25/2003 0:00,Shipped,3,8,2003,...,78934 Hillside Dr.,NaN,Pasadena,CA,90003,USA,NaN,Young,Julie,Medium
4,10159,49,100.00,14,5205.27,10/10/2003 0:00,Shipped,4,10,2003,...,7734 Strong St.,NaN,San Francisco,CA,NaN,USA,NaN,Brown,Julie,Medium


In [15]:
sd.duplicated('ORDERNUMBER')

0       False
1       False
2       False
3       False
4       False
        ...  
2818     True
2819     True
2820     True
2821     True
2822     True
Length: 2823, dtype: bool

In [18]:
sd.drop_duplicates('ORDERNUMBER', inplace=True)
sd.duplicated('ORDERNUMBER')

0       False
1       False
2       False
3       False
4       False
        ...  
2358    False
2532    False
2554    False
2692    False
2733    False
Length: 307, dtype: bool

In [20]:
sd['QUANTITYORDERED'].dropna()

0       30
1       34
2       41
3       45
4       49
        ..
2358    29
2532    32
2554    23
2692    36
2733    45
Name: QUANTITYORDERED, Length: 307, dtype: int64

In [23]:
sd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 307 entries, 0 to 2733
Data columns (total 25 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ORDERNUMBER       307 non-null    int64  
 1   QUANTITYORDERED   307 non-null    int64  
 2   PRICEEACH         307 non-null    float64
 3   ORDERLINENUMBER   307 non-null    int64  
 4   SALES             307 non-null    float64
 5   ORDERDATE         307 non-null    object 
 6   STATUS            307 non-null    object 
 7   QTR_ID            307 non-null    int64  
 8   MONTH_ID          307 non-null    int64  
 9   YEAR_ID           307 non-null    int64  
 10  PRODUCTLINE       307 non-null    object 
 11  MSRP              307 non-null    int64  
 12  PRODUCTCODE       307 non-null    object 
 13  CUSTOMERNAME      307 non-null    object 
 14  PHONE             307 non-null    object 
 15  ADDRESSLINE1      307 non-null    object 
 16  ADDRESSLINE2      31 non-null     object 
 

In [24]:
sd = sd.drop(['PHONE','ADDRESSLINE1','ADDRESSLINE2','STATE','POSTALCODE'], axis = 1)

In [27]:
#Changing the data type of variable 'ORDERDATE' from object to datetime
sd['ORDERDATE'] = pd.to_datetime(sd['ORDERDATE'])

In [28]:
sd.head()

,ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,PRODUCTLINE,MSRP,PRODUCTCODE,CUSTOMERNAME,CITY,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE
0,10107,30,95.70,2,2871.00,2003-02-24,Shipped,1,2,2003,Motorcycles,95,S10_1678,Land of Toys Inc.,NYC,USA,NaN,Yu,Kwai,Small
1,10121,34,81.35,5,2765.90,2003-05-07,Shipped,2,5,2003,Motorcycles,95,S10_1678,Reims Collectables,Reims,France,EMEA,Henriot,Paul,Small
2,10134,41,94.74,2,3884.34,2003-07-01,Shipped,3,7,2003,Motorcycles,95,S10_1678,Lyon Souveniers,Paris,France,EMEA,Da Cunha,Daniel,Medium
3,10145,45,83.26,6,3746.70,2003-08-25,Shipped,3,8,2003,Motorcycles,95,S10_1678,Toys4GrownUps.com,Pasadena,USA,NaN,Young,Julie,Medium
4,10159,49,100.00,14,5205.27,2003-10-10,Shipped,4,10,2003,Motorcycles,95,S10_1678,Corporate Gift Ideas Co.,San Francisco,USA,NaN,Brown,Julie,Medium


In [29]:
quant_vars = ['QUANTITYORDERED','PRICEEACH','SALES','MSRP']
sd[quant_vars].describe()

,QUANTITYORDERED,PRICEEACH,SALES,MSRP
count,307.000000,307.000000,307.000000,307.000000
mean,35.205212,93.351596,4588.486906,138.560261
std,10.431550,14.912985,2142.986902,45.667441
min,6.000000,34.910000,553.950000,35.000000
25%,27.000000,96.915000,3010.500000,102.000000
50%,35.000000,100.000000,4196.800000,136.000000
75%,43.000000,100.000000,5855.460000,170.000000
max,97.000000,100.000000,12001.000000,214.000000


In [31]:
sd.to_csv('sd.csv')